# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 5 2022 10:28AM,241066,15,PNC370948A,"Person & Covey, Inc.",Released
1,Jul 5 2022 10:28AM,241066,15,PNC370949A,"Person & Covey, Inc.",Released
2,Jul 5 2022 10:28AM,241066,15,PNC370950A,"Person & Covey, Inc.",Released
3,Jul 5 2022 10:28AM,241066,15,PNC370951A,"Person & Covey, Inc.",Released
4,Jul 5 2022 10:28AM,241066,15,PNC370953A,"Person & Covey, Inc.",Released
5,Jul 5 2022 10:28AM,241066,15,PNC370954A,"Person & Covey, Inc.",Released
6,Jul 5 2022 10:28AM,241066,15,PNC370955A,"Person & Covey, Inc.",Released
7,Jul 5 2022 10:26AM,241074,10,0085829587,ISDIN Corporation,Released
8,Jul 5 2022 10:26AM,241074,10,0085829588,ISDIN Corporation,Released
9,Jul 5 2022 10:26AM,241074,10,0085829507,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
50,241067,Released,40
51,241068,Released,1
52,241069,Released,55
53,241070,Released,49
54,241074,Released,41


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
241067,NaN,40.0
241068,NaN,1.0
241069,NaN,55.0
241070,NaN,49.0
241074,NaN,41.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241004,0.0,59.0
241005,0.0,54.0
241006,0.0,55.0
241009,0.0,60.0
241010,0.0,60.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241004,0,59
241005,0,54
241006,0,55
241009,0,60
241010,0,60


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241004,0,59
1,241005,0,54
2,241006,0,55
3,241009,0,60
4,241010,0,60


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241004,,59
1,241005,,54
2,241006,,55
3,241009,,60
4,241010,,60


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 5 2022 10:28AM,241066,15,"Person & Covey, Inc."
7,Jul 5 2022 10:26AM,241074,10,ISDIN Corporation
48,Jul 5 2022 10:26AM,241070,10,ISDIN Corporation
97,Jul 5 2022 10:26AM,241069,10,ISDIN Corporation
152,Jul 5 2022 10:26AM,241067,10,ISDIN Corporation
192,Jul 5 2022 10:23AM,241065,15,"Laser Pharmaceuticals, LLC"
208,Jul 5 2022 10:23AM,241064,15,"Carwin Pharmaceutical Associates, LLC"
210,Jul 5 2022 10:17AM,241068,10,Bio-PRF
211,Jul 5 2022 10:15AM,241063,15,"Brookfield Pharmaceuticals, LLC"
250,Jul 5 2022 9:59AM,241061,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 5 2022 10:28AM,241066,15,"Person & Covey, Inc.",,7
1,Jul 5 2022 10:26AM,241074,10,ISDIN Corporation,,41
2,Jul 5 2022 10:26AM,241070,10,ISDIN Corporation,,49
3,Jul 5 2022 10:26AM,241069,10,ISDIN Corporation,,55
4,Jul 5 2022 10:26AM,241067,10,ISDIN Corporation,,40
5,Jul 5 2022 10:23AM,241065,15,"Laser Pharmaceuticals, LLC",,16
6,Jul 5 2022 10:23AM,241064,15,"Carwin Pharmaceutical Associates, LLC",,2
7,Jul 5 2022 10:17AM,241068,10,Bio-PRF,,1
8,Jul 5 2022 10:15AM,241063,15,"Brookfield Pharmaceuticals, LLC",,39
9,Jul 5 2022 9:59AM,241061,10,ISDIN Corporation,,60


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 10:28AM,241066,15,"Person & Covey, Inc.",7,
1,Jul 5 2022 10:26AM,241074,10,ISDIN Corporation,41,
2,Jul 5 2022 10:26AM,241070,10,ISDIN Corporation,49,
3,Jul 5 2022 10:26AM,241069,10,ISDIN Corporation,55,
4,Jul 5 2022 10:26AM,241067,10,ISDIN Corporation,40,
5,Jul 5 2022 10:23AM,241065,15,"Laser Pharmaceuticals, LLC",16,
6,Jul 5 2022 10:23AM,241064,15,"Carwin Pharmaceutical Associates, LLC",2,
7,Jul 5 2022 10:17AM,241068,10,Bio-PRF,1,
8,Jul 5 2022 10:15AM,241063,15,"Brookfield Pharmaceuticals, LLC",39,
9,Jul 5 2022 9:59AM,241061,10,ISDIN Corporation,60,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 10:28AM,241066,15,"Person & Covey, Inc.",7,
1,Jul 5 2022 10:26AM,241074,10,ISDIN Corporation,41,
2,Jul 5 2022 10:26AM,241070,10,ISDIN Corporation,49,
3,Jul 5 2022 10:26AM,241069,10,ISDIN Corporation,55,
4,Jul 5 2022 10:26AM,241067,10,ISDIN Corporation,40,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 10:28AM,241066,15,"Person & Covey, Inc.",7.0,NaN
1,Jul 5 2022 10:26AM,241074,10,ISDIN Corporation,41.0,NaN
2,Jul 5 2022 10:26AM,241070,10,ISDIN Corporation,49.0,NaN
3,Jul 5 2022 10:26AM,241069,10,ISDIN Corporation,55.0,NaN
4,Jul 5 2022 10:26AM,241067,10,ISDIN Corporation,40.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 5 2022 10:28AM,241066,15,"Person & Covey, Inc.",7.0,0.0
1,Jul 5 2022 10:26AM,241074,10,ISDIN Corporation,41.0,0.0
2,Jul 5 2022 10:26AM,241070,10,ISDIN Corporation,49.0,0.0
3,Jul 5 2022 10:26AM,241069,10,ISDIN Corporation,55.0,0.0
4,Jul 5 2022 10:26AM,241067,10,ISDIN Corporation,40.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,10123506,1662.0,114.0
12,723140,81.0,1.0
15,1205317,84.0,0.0
19,482099,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,10123506,1662.0,114.0
1,12,723140,81.0,1.0
2,15,1205317,84.0,0.0
3,19,482099,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,1662.0,114.0
1,12,81.0,1.0
2,15,84.0,0.0
3,19,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,1662.0
1,12,Released,81.0
2,15,Released,84.0
3,19,Released,1.0
4,10,Executing,114.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19
Status,,,,
Executing,114.0,1.0,0.0,1.0
Released,1662.0,81.0,84.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19
0,Executing,114.0,1.0,0.0,1.0
1,Released,1662.0,81.0,84.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19
0,Executing,114.0,1.0,0.0,1.0
1,Released,1662.0,81.0,84.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()